In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import time

import torch
from torch.cuda import amp
from torch.utils.data import Dataset, DataLoader
import gc

from transformers import BartForConditionalGeneration,AutoTokenizer, BartConfig
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup

from dataset import KoBARTSummaryDataset

In [4]:
data_path='/content/drive/MyDrive/Dacon/한국어 생성요약/data/235813_AI 기반 회의 녹취록 요약 경진대회_data/'
model_path='/content/drive/MyDrive/Dacon/한국어 생성요약/LG_model/'
sub_path='/content/drive/MyDrive/Dacon/한국어 생성요약/LG_sub/'

# Train

In [8]:
tokenizer=AutoTokenizer.from_pretrained("hyunwoongko/kobart")

args={}
args['train_path']=data_path+'train_evi_concat_final.csv'
args['test_path']=data_path+'test_evi_final.csv'
args['weight_path']=model_path
args['max_len']=1024
args['batch_size']=4
args['num_workers']=2
args['max_epochs']=2

In [6]:
train_data = KoBARTSummaryDataset(args['train_path'],tok=AutoTokenizer.from_pretrained("hyunwoongko/kobart"),max_len=args['max_len'])

train_dataloader = DataLoader(train_data,
                              batch_size=args['batch_size'], 
                              shuffle=True,
                              num_workers=args['num_workers'], 
                              pin_memory=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [7]:
next(iter(train_data))

{'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.int32),
 'decoder_attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0], dtype=torch.int32),
 'decoder_input_ids': tensor([    0, 12147, 14745,   254, 13737, 14480, 12258, 16810, 13737, 17711,
         13737, 15953, 12612, 24124, 17896, 23129,   245,     1,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,  

In [ ]:
model=BartForConditionalGeneration.from_pretrained("hyunwoongko/kobart").to(device)


optimizer = AdamW(model.parameters(),lr=2e-5, weight_decay=1e-4,correct_bias=False)
scaler = amp.GradScaler()


for epoch_i in range(0, args['max_epochs']):
  
  # ========================================
  #               Training
  # ========================================

  print("")
  print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, args['max_epochs']))
  model.train()
  t0 = time.time()
  total_train_loss = 0
  total_batch=len(train_dataloader)

  for i, batch in enumerate(train_dataloader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    decoder_input_ids = batch['decoder_input_ids'].to(device)
    decoder_attention_mask = batch['decoder_attention_mask'].to(device)
    labels = batch['labels'].to(device)
    
    optimizer.zero_grad()

    with amp.autocast():
      loss = model(input_ids=input_ids,
                   attention_mask=attention_mask,
                   decoder_input_ids=decoder_input_ids,
                   decoder_attention_mask=decoder_attention_mask,
                   labels=labels)[0]
      
    total_train_loss += loss.item()

    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
      
    training_time = time.time() - t0

    print(f"\rTotal Batch {i+1}/{total_batch} , elapsed time : {training_time/(i+1):.1f}s , train_loss : {total_train_loss/(i+1):.2f}", end='')
  print("")

  model.save_pretrained(os.path.join(args['weight_path'], "kobart_title_evi_concat_final_epoch_{}".format(epoch_i+1)))

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]


======== Epoch 1 / 2 ========
Total Batch 1497/1497 , elapsed time : 0.4s , train_loss : 0.75

======== Epoch 2 / 2 ========
Total Batch 1497/1497 , elapsed time : 0.4s , train_loss : 0.41


# Generation

+ Generate(sets, max_length=150, no_repeat_ngram_size=3)

In [11]:
# test data load
test_data = KoBARTSummaryDataset(args['test_path'],tok=AutoTokenizer.from_pretrained("hyunwoongko/kobart"),max_len=args['max_len'],infer=True)

test_dataloader = DataLoader(test_data,
                              batch_size=args['batch_size'], 
                              shuffle=False,
                              num_workers=args['num_workers'], 
                              pin_memory=True)

In [ ]:
# prediction
test_sum=[]
test_model=BartForConditionalGeneration.from_pretrained(os.path.join(args['weight_path'], "kobart_title_evi_concat_final_epoch_2")).to(device)

for i,batch in tqdm(enumerate(test_dataloader)):
  sets=batch['input_ids'].to(device)
  batch_sum=test_model.generate(sets, max_length=150,no_repeat_ngram_size=3)
  test_sum=[*test_sum,*batch_sum]

127it [01:28,  1.43it/s]


In [ ]:
test_sum_sent=[tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in test_sum]

sub=pd.read_csv(data_path+'sample_submission.csv')
sub['summary']=test_sum_sent

In [ ]:
sub.to_csv('kobart_epoch2_final_g3.csv',index=False)

# View Sub

In [12]:
sub=pd.read_csv(sub_path+'kobart_epoch2_final_g3.csv')
test_real=pd.read_csv(data_path+'test_evi_final.csv')

In [18]:
id=35
print('Context:\n')
display(test_real['title_context'][id])

print('\nsummary:\n')
display(sub['summary'][id])

Context:



'제76회 본회의 제1차: 의사일정 제1항 의장·부의장 선거를 상정합니다. 투표결과는 집계가 끝나는 대로 말씀드리도록 하겠습니다. (개 표) 투표결과를 발표하겠습니다. 총 투표수 9표 중에서 이준구 의원 6표, 김우식 의원 3표로 결정이 났습니다. 1차 투표에서 과반수 득표를 얻으신 이준구 의원이 지방자치법 제42조 제1항 음성군의회 회의규칙 제8조 제1항의 규정에 의하여 의장에 당선되었음을 선포합니다. 투표결과는 집계가 끝나는 대로 말씀드리도록 하겠습니다. 투표결과를 발표하겠습니다. 총 투표수 9표 중에서 고재협 의원이 7표, 무효투표 2표가 나왔습니다. 1차 투표에서 과반수 득표를 얻으신 고재협 의원이 지방자치법 제42조 제1항 및 음성군의회 회의규칙 제8조 제1항의 규정에 의하여 부의장에 당선되었음을 선포합니다.'


summary:



'의장 선거는 총 9표 중에서 이준구 의원 6표, 김우식 의원 3표로 이준구 의원이 당선. 부의장 선거 투표는 총 8표 중에서 고재협 의원 7표, 무효표 2표로 고재협의 의원 당선.'

In [26]:
id=3
print('Context:\n')
display(test_real['title_context'][id])

print('\nsummary:\n')
display(sub['summary'][id])

Context:



'제235회 본회의 제1차: 의사일정 제3항, 예산결산특별위원회 구성의 건을 상정합니다. 예산결산특별위원회 구성은 의원님들이 사전에 양해하여 주신 대로 제가 특별위원회 위원을 지명하도록 하겠습니다. 먼저 예산결산특별위원회 위원은 손수종 의원님, 이한철 의원님, 남궁유 의원님, 조천희 의원님, 손달섭 의원님, 이대웅 의원님, 김순옥 의원님으로 구성하고자 합니다. 그리고 본 특별위원회 운영기간은 6월 25일 하루 동안 운영하고자 하며, 2011년도 예비비 지출 승인안, 2011회계 세입ㆍ세출 결산 승인안, 2011년도 회계 기금운용 성과분석보고 등을 회부하여 의사일정에 따라 심사토록 하고자 합니다. 제가 제의한 예산결산특별위원회 구성안에 대하여 이의가 없으므로 가결되었음을 선포합니다.'


summary:



'예산결산특별위원회 위원은 손수종 의원, 이한철 의원, 남궁유 의원, 조천희 의원, 손달섭 의원, 이대웅 의원, 김순옥 의원으로 구성함. 특별위원회는 6월 25일 하루동안 2011년도 예비비 지출 승인안과 2011회계 세입 세출 결산 승인안, 2011년도 회계 기금운용 성과분석보고 등을 의사일정에 따라 심사하고자 구성함'